In [2]:
from fastapi import FastAPI
import pickle
import numpy as np
from pydantic import BaseModel
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
import pickle

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

X_train = train_df.drop('Cover_Type', axis=1)
y_train = train_df['Cover_Type']

X_test = test_df

lgb_model = LGBMClassifier(random_state=42)
lgb_model.fit(X_train, y_train)

mlp_model = MLPClassifier(random_state=42)
mlp_model.fit(X_train, y_train)

ensemble_model = VotingClassifier(estimators=[
    ('lgb', lgb_model),
    ('mlp', mlp_model)
], voting='hard')
ensemble_model.fit(X_train, y_train)

lgb_predictions = lgb_model.predict(X_test)
mlp_predictions = mlp_model.predict(X_test)
ensemble_predictions = ensemble_model.predict(X_test)

# You can evaluate the models using any appropriate metrics
# For this example, let's just print the predicted classes
print("LightGBM Predictions:", lgb_predictions)
print("MLP Predictions:", mlp_predictions)
print("Ensemble Predictions:", ensemble_predictions)

with open('lgb_model.pkl', 'wb') as f:
    pickle.dump(lgb_model, f)

with open('mlp_model.pkl', 'wb') as f:
    pickle.dump(mlp_model, f)

with open('ensemble_model.pkl', 'wb') as f:
    pickle.dump(ensemble_model, f)

with open('lgb_model.pkl', 'rb') as f:
    model1 = pickle.load(f)

with open('mlp_model.pkl', 'rb') as f:
    model2 = pickle.load(f)

with open('ensemble_model.pkl', 'rb') as f:
    ensemble_model = pickle.load(f)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000131 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2426
[LightGBM] [Info] Number of data points in the train set: 15120, number of used features: 46
[LightGBM] [Info] Start training from score -1.945910
[LightGBM] [Info] Start training from score -1.945910
[LightGBM] [Info] Start training from score -1.945910
[LightGBM] [Info] Start training from score -1.945910
[LightGBM] [Info] Start training from score -1.945910
[LightGBM] [Info] Start training from score -1.945910
[LightGBM] [Info] Start training from score -1.945910
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000508 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2426
[LightGBM] [Info] Number of data points in the train set: 15120, nu

In [3]:
app = FastAPI()

class InputFeatures(BaseModel):
    elevation: float
    aspect: float
    slope: float
    horizontal_distance_to_hydrology: float
    vertical_distance_to_hydrology: float
    horizontal_distance_to_roadways: float
    hillshade_9am: float
    hillshade_noon: float
    hillshade_3pm: float
    horizontal_distance_to_fire_points: float
    wilderness_area: list[int]
    soil_type: list[int]

class PredictionResult(BaseModel):
    predicted_class: int

@app.get("/")
def read_root():
    return {"message": "Welcome to the Forest Cover Type Prediction API"}

@app.get("/version")
def get_version():
    return {"version": "1.0"}

@app.post("/predict1", response_model=PredictionResult)
def predict_top_classifier(data: InputFeatures):
    features = np.array([list(data.dict().values())[:-2]]).astype(np.float32)
    prediction = model1.predict(features)
    return {"predicted_class": int(prediction[0])}

@app.post("/predict2", response_model=PredictionResult)
def predict_second_best_classifier(data: InputFeatures):
    features = np.array([list(data.dict().values())[:-2]]).astype(np.float32)
    prediction = model2.predict(features)
    return {"predicted_class": int(prediction[0])}

@app.post("/predict3", response_model=PredictionResult)
def predict_ensemble_classifier(data: InputFeatures):
    features = np.array([list(data.dict().values())[:-2]]).astype(np.float32)
    prediction = ensemble_model.predict(features)
    return {"predicted_class": int(prediction[0])}

In [6]:
import uvicorn

if __name__ == "__main__":
    uvicorn.run(app, --host 0.0.0.0 --port 8000)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2488580257.py, line 4)